In [53]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Code/ABIDE-I-slurm_jobs'

In [54]:
import os
import shutil

In [55]:
indir = '../../../data/ABIDE/RawDataBIDS/'
folders = [f for f in os.listdir(indir) if os.path.isdir(os.path.join(indir,f))]
folders.sort()
len(folders)

24

In [56]:
#sum(subCount)

In [57]:
subCount = list()
c = -1
for folder in folders:
    c+=1
    nsubs = len([cont for cont in os.listdir(os.path.join(indir,folder)) if all((cont.startswith('sub-'),os.path.isdir(os.path.join(indir,folder,cont))))])
    subCount.append(nsubs)
    #print(f'{folder} | {nsubs}'.rjust(20,"#"))
    print(f'{c:02d} |'.rjust(5)  + f'{folder}'.center(10) + f'| {nsubs:03d}'.rjust(5))

 00 |  CMU_a   | 014
 01 |  CMU_b   | 013
 02 | Caltech  | 038
 03 |   KKI    | 055
 04 | Leuven_1 | 029
 05 | Leuven_2 | 035
 06 | MaxMun_a | 018
 07 | MaxMun_b | 006
 08 | MaxMun_c | 017
 09 | MaxMun_d | 016
 10 |   NYU    | 184
 11 |   OHSU   | 028
 12 |   Olin   | 036
 13 |   Pitt   | 057
 14 |   SBL    | 030
 15 |   SDSU   | 036
 16 | Stanford | 040
 17 | Trinity  | 049
 18 |  UCLA_1  | 082
 19 |  UCLA_2  | 027
 20 |   UM_1   | 110
 21 |   UM_2   | 035
 22 |   USM    | 101
 23 |   Yale   | 056


In [285]:
i = -1

In [286]:
i+=1
folder = folders[i]
n = subCount[i]
print(i)
print(folder)
print(n)

0
CMU_a
14


In [277]:
if not os.path.isdir(f'fmriprep_ABIDEI-{folder}'):
    os.mkdir(f'fmriprep_ABIDEI-{folder}')

In [278]:
text = '''#!/bin/bash

#SBATCH --job-name=job_fmriprep_ABIDEI-{folder}
#SBATCH --output=./fmriprep_ABIDEI-{folder}/output_fmriprep_ABIDEI-{folder}_%a
#SBATCH --error=./fmriprep_ABIDEI-{folder}/error_fmriprep_ABIDEI-{folder}_%a

#SBATCH --ntasks=1
#SBATCH --time=10:00:00
#SBATCH --mem=32gb

#SBATCH --array=0-{n}


module load singularity/

date

#rm -rf ./fmriprep_ABIDEI-{folder}/output_*
#rm -rf ./fmriprep_ABIDEI-{folder}/error_*

cd $HOME

data_dir=$HOME'/data/ABIDE/RawDataBIDS/{folder}/'
sing_image=$HOME/fmriprep.simg

cd $data_dir
files=(sub*)

cd $HOME

output_dir=$HOME/data/ABIDE/Derivatives/fmriprep/

fs_lic=$HOME'/fs_licence.txt'

sub=${{files[$SLURM_ARRAY_TASK_ID]}} #;echo $sub

echo $HOME
echo $data_dir
echo $output_dir
echo $sub

singularity run --bind /data/aglinska/scratch:/scratch --cleanenv $sing_image $data_dir $output_dir participant --participant-label $sub --fs-no-reconall --task-id 'rest' --fs-license-file $fs_lic --ignore slicetiming


'''

In [279]:
text = text.format(folder=folder,n=n)

In [280]:
with open(f'job_fmriprep_{folder}.slurm', 'w') as f:
    f.write(text)

In [281]:
bashCommand = f'sbatch job_fmriprep_{folder}.slurm'
print(bashCommand)

sbatch job_fmriprep_Yale.slurm


In [282]:
with open(f'command.sh', 'w') as f:
    f.write(bashCommand)

In [283]:
!chmod 777 command.sh

In [284]:
!./command.sh

Submitted batch job 270736
